# Review Notebook for Sense Filtering by Time and Provenance

This notebook builds on 4.1 (harvest senses with provenance) and 4.2 (harvest quotations for senses). The code in this notebook allows you to filter senses by (a) selecting seed senses (b) defining relations words should have to the seed senses.

Functions reviewed in this notebook:
- `filter_by_year_range` (helper)
- `select_senses_by_provenance`(helper)
- `filter_senses` (main)

Part of:
- `utils.dataset_download`

Creator: Kaspar Beelen

Reviewer(s):


In [1]:
!git branch

  19-machine-tagger
* 4-semantic-provenance
  44-kNN-BERT-baseline
  dev
  master


## Load libraries, data and set parameters

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [48]:
import pandas as pd
import json
from utils.dataset_download import *

In [5]:
lemma_id = 'machine_nn01'

In [6]:
# import API credentials
with open('../oed_experiments/oed_credentials.json') as f:
    auth = json.load(f)

In [24]:
df = pd.read_pickle(f'./data/extended_{lemma_id}.pickle')

In [8]:
df.head(3)

,categories,daterange,definition,first_use,id,lemma,main_current_sense,meta,notes,oed_reference,oed_url,part_of_speech,provenance,provenance_type,quotation_ids,semantic_class_ids,semantic_class_last_id,transitivity,word_id
0,"{'topic': [], 'usage': [['rare']], 'region': []}","{'end': None, 'start': 1545, 'obsolete': False...","A material or immaterial structure, esp. the f...",J. Schäfer,machine_nn01-38473945,machine,False,"{'created': 1904, 'revised': True, 'updated': ...",[],"machine, n., sense I.1a",https://www.oed.com/view/Entry/111850#eid38473945,NN,"[[machine_nn01-38473945, seed, machine_nn01]]",seed,"[machine_nn01-38473950, machine_nn01-38473961,...","[[1, 111290, 118635, 119024, 120162, 120172], ...","[120172, 120173]",None,machine_nn01
1,"{'topic': [['Military', 'Weaponry']], 'usage':...","{'end': None, 'start': 1583, 'obsolete': False...",A military engine or siege-tower. Cf. war mach...,Brian Melbancke,machine_nn01-38474233,machine,False,"{'created': 1904, 'revised': True, 'updated': ...",[],"machine, n., sense II.3",https://www.oed.com/view/Entry/111850#eid38474233,NN,"[[machine_nn01-38474233, seed, machine_nn01]]",seed,"[machine_nn01-38474243, machine_nn01-38474252,...","[[153072, 160439, 163207, 163208, 163377, 1633...",[163378],None,machine_nn01
2,"{'topic': [], 'usage': [], 'region': []}","{'end': 1707, 'start': 1595, 'obsolete': True,...",spec. A scheme or plot. Obsolete.,Elizabeth I,machine_nn01-38474097,machine,False,"{'created': 1904, 'revised': True, 'updated': ...",[],"machine, n., sense I.1b",https://www.oed.com/view/Entry/111850#eid38474097,NN,"[[machine_nn01-38474097, seed, machine_nn01]]",seed,"[machine_nn01-38474102, machine_nn01-38474122,...","[[1, 84689, 87987, 87988, 87989, 88083, 88109,...",[88126],None,machine_nn01


In [60]:
df_quotations = pd.read_pickle(f'./data/quotations_all_{lemma_id}.pickle')
df_quotations.shape

(203560, 12)

In [10]:
df_quotations.head()

,id,text,year,lemma,source,oed_url,word_id,sense_id,datestring,first_in_word,oed_reference,first_in_sense
0,pigmeat_nn01-13163366,"{'keyword': 'pig-meat', 'full_text': 'I was at...",1754,pigmeat,"{'title': 'Connoisseur', 'author': 'G. Colman'...",https://www.oed.com/view/Entry/237320#eid13163366,pigmeat_nn01,pigmeat_nn01-13163363,1754,True,"pigmeat, n., sense 1",True
1,pigmeat_nn01-13163379,"{'keyword': 'pig-meat', 'full_text': 'In short...",1784,pigmeat,"{'title': 'Year's Journey through Paix Bâs', '...",https://www.oed.com/view/Entry/237320#eid13163379,pigmeat_nn01,pigmeat_nn01-13163363,1784,False,"pigmeat, n., sense 1",False
2,pigmeat_nn01-13163399,"{'keyword': 'pig meat', 'full_text': 'It preve...",1817,pigmeat,"{'title': 'Parl. Deb.', 'author': None, 'gende...",https://www.oed.com/view/Entry/237320#eid13163399,pigmeat_nn01,pigmeat_nn01-13163363,1817,False,"pigmeat, n., sense 1",False
3,pigmeat_nn01-13163416,"{'keyword': 'pig meat', 'full_text': 'In most ...",1897,pigmeat,"{'title': 'Syst. Med.', 'author': 'T. C. Allbu...",https://www.oed.com/view/Entry/237320#eid13163416,pigmeat_nn01,pigmeat_nn01-13163363,1897,False,"pigmeat, n., sense 1",False
4,pigmeat_nn01-13163425,"{'keyword': 'pig meat', 'full_text': 'Beef tak...",1918,pigmeat,"{'title': 'Times', 'author': None, 'gender': N...",https://www.oed.com/view/Entry/237320#eid13163425,pigmeat_nn01,pigmeat_nn01-13163363,1918,False,"pigmeat, n., sense 1",False


# Run code

In [51]:
set(df[df.id.isin(['machine_nn01-38475835','machine_nn01-38475923'])].definition)

{'A conceptual, abstract, or theoretical mechanism or device; spec. a model or a mathematical abstraction of an existing or hypothetical computer. Cf. Turing machine n.',
 'Mechanics. Anything that transmits force or directs its application.'}

In [99]:
senses = filter_senses(df,
                       {'machine_nn01-38475835','machine_nn01-38475923'},
                       relations = ['seed','synonym','descendant','sibling'],
                       skip_synonyms=False,
                       start=1760, 
                       end=1920
                      )

# senses before filtering by date = 8383
# senses after filtering by date = 5918


# of seed senses 22 
# of synonyms 310 
# of branch senses 4990


# of seeds selected 1 
# of synonyms selected 8 
# of branches selected 10


In [54]:
len(senses)

29

In [57]:
df_source = pd.read_pickle("../data/extended_machine_nn01.pickle")

In [94]:
quotations = obtain_quotations_for_senses(df_quotations,
                                df_source,                  
                                senses,
                                start=1760,end=1920)
quotations.shape

(178, 13)

In [95]:
quotations.head()

,keyword,full_text,keyword_offset,title,author,gender,year,sense_id,daterange,provenance,provenance_type,relation_to_core_senses,relation_to_seed_senses
0,wedge,"The sharper the wedge, or the more acute its a...",16.0,Princ. Mech.,W. Emerson,male,1773,wedge_nn01-14811588,"{'end': None, 'start': 725, 'obsolete': False,...","[[214221, descendant, 214220]]",branch,"{machine_nn01-38475923, power_nn01-28687898}",{machine_nn01-38475923}
1,wedge,Forth goes the woodman..To wield the axe And d...,55.0,Task,W. Cowper,male,1785,wedge_nn01-14811588,"{'end': None, 'start': 725, 'obsolete': False,...","[[214221, descendant, 214220]]",branch,"{machine_nn01-38475923, power_nn01-28687898}",{machine_nn01-38475923}
2,wedges,The distended ropes..are then confined at the ...,57.0,Archit.,W. Newton,None,1789,wedge_nn01-14811588,"{'end': None, 'start': 725, 'obsolete': False,...","[[214221, descendant, 214220]]",branch,"{machine_nn01-38475923, power_nn01-28687898}",{machine_nn01-38475923}
3,wedges,The wedges employed to secure the rails in the...,4.0,Minutes Proc. Inst. Civil Engineers,None,None,1842,wedge_nn01-14811588,"{'end': None, 'start': 725, 'obsolete': False,...","[[214221, descendant, 214220]]",branch,"{machine_nn01-38475923, power_nn01-28687898}",{machine_nn01-38475923}
4,Wedge,"Wedge,..a small fastening for a door or window.",0.0,Dict. Trade Products,P. L. Simmonds,male,1858,wedge_nn01-14811588,"{'end': None, 'start': 725, 'obsolete': False,...","[[214221, descendant, 214220]]",branch,"{machine_nn01-38475923, power_nn01-28687898}",{machine_nn01-38475923}


In [96]:
set().union(*quotations.relation_to_core_senses)

{'energizer_nn01-1246574970',
 'generator_nn01-135539783',
 'machine_nn01-38475923',
 'machinepower_nn01-38476982',
 'mechanicaladvantage_nn01-37502752',
 'mover_nn01-35820685',
 'power_nn01-28687898',
 'primemover_nn01-28348676',
 'primummobile_nn01-28369573',
 'vice_nn02-15526247'}

In [97]:
set().union(*quotations.relation_to_seed_senses)

{'machine_nn01-38475923'}

# Inspect code

### Inspect `filter_by_year_range`

In [ ]:
def filter_by_year_range(dr: dict, target_start: int, target_end: int) -> bool:
    """
    Helper function that expects a datarange dictionary from the OED
    Is used for filter senses that are outside the historical scope 
    of the research. The date range is defined by the target_start and target_end
    arguments. If the date range of the sense has NO overlap with the
    target period, then return False, otherwise return True
    
    Arguments:
        dr (dict): daterange dict of OED
        target_start (int): start year of target period
        target_end (int): end year of target period
    
    Returns:
        return a boolean, True if there is overlap between
        the target period and the date range of the sense
    """
    # if there is not start date, set to 0
    if dr.get('start',None) is None:
        sense_start = 0
    else:
        sense_start = dr['start']
    
    
    # if there is no end date, set to 2021
    if dr.get('end',None) is None:
        sense_end = 2021
    else:
        sense_end = dr['end']
    
    # if there is an intersection between the target period and sense period empty
    # return True
    if set(range(sense_start,sense_end+1)).intersection(set(range(target_start,target_end+1))):
        return True
    
    # otherwise return False
    return False

In [ ]:
# number of senses selected
df['in_period'] = df.daterange.apply(filter_by_year_range, target_start=1760, target_end=1920)
print(sum(df.in_period))

In [ ]:
#print(df['in_period'][110:120])

In [ ]:
print(df.iloc[0][['daterange','in_period']])
print(df.iloc[2][['daterange','in_period']])
print(df.iloc[70][['daterange','in_period']])
print(df.iloc[88][['daterange','in_period']])
print(df.iloc[89][['daterange','in_period']])

### Inspect `select_senses_by_provenance`

In [ ]:
def select_senses_by_provenance(sub_df: pd.DataFrame, 
                                item_ids: set, 
                                relations: list) -> tuple:
    """Helper function that given a subsection of a dataframe filters senses based
    on a set of target sense ids and relations. This function requires a dataframe created
    by the extend_from_lemma function.
    
    Arguments:
        sub_df (pd.DataFrame): slice of a pd.DataFrame
        item_ids (set): include senses related to these items 
                        these can be sense ids or semantic class ids
        relations (list): filter based on these relations 
                          options are: seed, synonyms, sibling, descedant
        
    Returns:
        a tuple that contains a list with position indices and a list with items
    """
    
    indices, items = set(),set()
    
    for i, row in sub_df.iterrows():
        for oed_id, relation, prov_id in row.provenance:
            # if the provenance and relation match to the arguments
            # add the items and position to the respective lists
            if (prov_id in item_ids) and (relation in relations):
                indices.add(i) ; items.add(oed_id)
                
    return list(indices), list(items)

In [ ]:
sel_indices, selected = select_senses_by_provenance(df, {'machine_nn01-38475835','machine_nn01-38475923'},
                                                        ['synonym'])

In [ ]:
selected

### Inspect `filter_senses`

In [ ]:
def filter_senses(df, sense_ids:set, 
                      relations:list, 
                      start:int, 
                      end:int,
                      verbose=True) -> set:
    """
    Main function that filter sense by a give date range 
    and set of seed senses with provenace relations. 
    The seeds sense are selected from the lemma dataframe
    used as starting point for harvesting. Builds on dataframe created 
    by the extend_from_lemma function.
    
    Returns selected senses as a set. 
    
    Arguments:
        df (pd.DataFrame): main dataframe created by the extend_from_lemma
        senses_ids (set): seeds senses from the lemma used for filtering
        relations (list): filter based on these relations
        start (int): beginning of target period
        end (int): end of target period
        verbose (bool): print outcomes of intermediate steps
    
    Returns:
        set with senses
    """
    print("# senses before filtering by date =", df.shape[0])
    df = df[df.daterange.apply(filter_by_year_range, target_start=start, target_end=end)]
    print("# senses after filtering by date =", df.shape[0])
    
    
    seeds = df[df['provenance_type'] == "seed"].reset_index(inplace=False)
    # select words retrieved as synonyms
    # exclude those that already appear in the seed dataframe
    # reset index after selection
    synonyms = df[(df['provenance_type'] == "synonym") & (~df.id.isin(seeds.id))
                     ].reset_index(inplace=False)
    
    # select words retrieved as a branch of the synonym or a seed sense
    # exclude those that already appear as seed or synonym
    branches = df[(df['provenance_type'] == "branch") & (~df.id.isin(set(seeds.id).union(set(synonyms.id))))
                      ].reset_index(inplace=False)
    
    print("\n\n# of seed senses", seeds.shape[0],
          "\n# of synonyms", synonyms.shape[0],
          "\n# of branch senses", branches.shape[0])

    if "seed" in relations:
        seeds_selected = set(seeds[seeds.id.isin(sense_ids)].id)

    if "synonym" in relations:
        syn_sel_indices, synonyms_selected = select_senses_by_provenance(synonyms,sense_ids,relations)
    
    # as branches are retrieved by semantic class id, we get the semantic class ids 
    # of the seed AND synonyms senses
    select_seed_semantic_class_id = seeds[seeds.id.isin(seeds_selected)].semantic_class_last_id
    select_seed_semantic_class_id = set().union(*map(set,select_seed_semantic_class_id))
    
    select_synonyms_semantic_class_id = synonyms[synonyms.id.isin(synonyms_selected)].semantic_class_last_id
    select_synonyms_semantic_class_id = set().union(*map(set,select_synonyms_semantic_class_id))
    
    selected_semantic_class_id = set(select_seed_semantic_class_id).union(set(select_synonyms_semantic_class_id))
    
    branch_sel_indices, branches_selected = select_senses_by_provenance(branches,selected_semantic_class_id,relations)
    
    
    senses = set(branches.iloc[branch_sel_indices].id # for the branches we return the sense ids not the semantic class ids
               ).union(set(synonyms.iloc[syn_sel_indices].id)
                        ).union(set(seeds_selected))
    if verbose:
        print('\n\n# of seeds selected', len(seeds_selected),
              '\n# of synonyms selected', len(syn_sel_indices),
              '\n# of branches selected', len(branches_selected))
    return senses

In [ ]:
senses = filter_senses(df,
                       {'machine_nn01-38475835','machine_nn01-38475923'},
                       ['seed','synonym','descendant'],
                       start=1760, 
                       end=1920
                      )

In [ ]:
senses

### Inspect `obtain_quotations_for_senses`

In [20]:
def obtain_quotations_for_senses(
                      df_quotations:  pd.DataFrame,
                      senses: set,
                      start:int,
                      end: int
                    ) -> pd.DataFrame:
    """Create a dataframe with quotations and their metadata for 
    a selected set of senses. This function builds on
    harvest_quotations_by_sense_id.
    
    Arguments:
        df_quotations: dataframe with quotations, created using harvest_quotations_by_sense_id
        senses (set): set of senses for which we want to obtain quotations
        
    Returns:
        pd.DataFrame with quotations
        
    """
    df = pd.concat([
        pd.DataFrame.from_records(df_quotations.text.values),
        pd.DataFrame.from_records(df_quotations.source.values)
            ], axis=1)
    df['year'] = df_quotations['year']
    df['sense_id'] = df_quotations['sense_id']
    df = df[df.sense_id.isin(senses)]
    df = df[(start <= df.year) & (df.year <= end)]
    
    df.drop_duplicates(inplace=True)
    
    return df



In [21]:
senses

{'accumulator_nn01-34592667',
 'commutator_nn01-8782500',
 'compressor_nn01-8608218',
 'compressor_nn01-8608248',
 'dynamo_nn01-5878672',
 'energizer_nn01-1246574970',
 'generator_nn01-135539783',
 'heatexchanger_nn01-1908252',
 'inductordisk_nn01-1192999040',
 'inductorplate_nn01-1192999070',
 'lever_nn01-39377191',
 'leverage_nn01-39378231',
 'machine_nn01-38475923',
 'machinepower_nn01-38476982',
 'mechanicaladvantage_nn01-37502752',
 'mover_nn01-35820685',
 'pair_nn01-32480956',
 'pairing_nn02-32485248',
 'peritrochium_nn01-30868432',
 'power_nn01-28687898',
 'primemover_nn01-28348676',
 'primummobile_nn01-28369573',
 'prise_nn03-28455147',
 'purchase_nn01-27522250',
 'rocker_nn01-25157825',
 'screw_nn01-24083320',
 'screwcompressor_nn01-24085973',
 'wedge_nn01-14811588',
 'wheel_nn01-14697935'}

In [22]:
quotations = obtain_quotations_for_senses(df_quotations,senses,1760,1920)
quotations.head()

,keyword,full_text,keyword_offset,title,author,gender,year,sense_id
9438,wedge,"The sharper the wedge, or the more acute its a...",16.0,Princ. Mech.,W. Emerson,male,1773,wedge_nn01-14811588
9440,wedge,Forth goes the woodman..To wield the axe And d...,55.0,Task,W. Cowper,male,1785,wedge_nn01-14811588
9441,wedges,The distended ropes..are then confined at the ...,57.0,Archit.,W. Newton,None,1789,wedge_nn01-14811588
9457,wedges,The wedges employed to secure the rails in the...,4.0,Minutes Proc. Inst. Civil Engineers,None,None,1842,wedge_nn01-14811588
9463,Wedge,"Wedge,..a small fastening for a door or window.",0.0,Dict. Trade Products,P. L. Simmonds,male,1858,wedge_nn01-14811588


## Fin.